In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q torch transformers datasets trl accelerate
!pip install -q flash-attn --no-build-isolation

import os
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('HF_TOKEN')
login(token=hf_token)
os.environ['HF_TOKEN'] = hf_token

from datasets import load_dataset

dataset_file_name = "food_data.jsonl"
dataset = load_dataset("json", data_files=dataset_file_name, split="train")

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'right'

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="flash_attention_2"
)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)
print(dataset["text"][0])

from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./phi-3-full-finetune",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    save_strategy="epoch",
    save_total_limit=2,
    logging_steps=10,
    bf16=True,
    gradient_checkpointing=True,
    max_grad_norm=1.0,
    report_to="none",
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

print("Training starting...")
trainer.train()

# Save model locally
print("\nSaving model locally...")
trainer.save_model("./phi-3-full-finetune-final")
tokenizer.save_pretrained("./phi-3-full-finetune-final")

# Copy model specific files
import shutil
from pathlib import Path

special_files = ["modeling_phi3.py", "configuration_phi3.py"]
for file in special_files:
    src = Path(model.config._name_or_path) / file
    if src.exists():
        shutil.copy(src, f"./phi-3-full-finetune-final/{file}")

print("Local save complete.")

# Copy to Google Drive
print("\nCopying to Google Drive...")
drive_path = "/content/drive/MyDrive/phi-3-food-recipes-final"

if os.path.exists(drive_path):
    shutil.rmtree(drive_path)

shutil.copytree("./phi-3-full-finetune-final", drive_path)
print(f"Model saved to Google Drive: {drive_path}")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 32.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Generating train split: 0 examples [00:00, ? examples/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Truncating train dataset:   0%|          | 0/24720 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Training starting...


Step,Training Loss
10,1.500800
20,1.471600
30,1.352200
40,1.241300
50,1.169000
60,1.068900
70,1.014200
80,0.946900
90,0.947700
100,0.899100



Saving model locally...
Local save complete.

Copying to Google Drive...
Model saved to Google Drive: /content/drive/MyDrive/phi-3-food-recipes-final


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "/content/drive/MyDrive/trainingphi3"

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    local_files_only=True
)

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True,
    local_files_only=True
)

print("Model loaded successfully!\n")

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Can you make a dinner, easy dish with chicken, rice, vegetables?

### Input:


### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=500,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
    use_cache=False
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded successfully!

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Can you make a dinner, easy dish with chicken, rice, vegetables?

### Input:


### Response:
Here you go:

**Brown Rice Chicken**

Ingredients:
- chicken
- rice
- vegetables
- onion
- olive oil
- garlic
- salt
- pepper
- basil
- bay leaves

**Instructions:**
1. Heat oil in large saucepan
2. Add chicken , onion and garlic
3. Cook , stirring occasionally , until chicken is browned
4. Add remaining ingredients and bring to boil
5. Reduce heat , cover and simmer for 30 minutes
6. Remove bay leaves
7. Cook uncovered for 10 minutes or until liquid is absorbed
